In [0]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from sklearn.tree import DecisionTreeClassifier

import numpy as np

sns.set(style="whitegrid")

# 0. Etapas da Sessão 01 e 02
Aqui nós colocamos todas as etapas das Sessões anterior uma única célula, para ver a explicação de cada etapa você pode olhar o notebook da Sessão 01 e da Sessão 02 em https://github.com/widsrecife/live-coding


In [0]:
# Ler dados de um CSV e coloca em um dataframe

df = pd.read_csv("https://raw.githubusercontent.com/widsrecife/dados/master/Sessao_1/Attribute%20DataSet.csv")

# Removendo linhas que não têm valor em Season

df = df.dropna(subset=["Season"])

# O pandas tem várias formas de selecionar linhas e colundas ([], .loc, .iloc)
#   aqui vamos usar o [] (indexing operator) onde a gente passa uma lista com as colunas que queremos selecionar
colunas_utilizadas = ["Style","NeckLine","SleeveLength","Material","FabricType","Decoration"]

df_colunas_utilizadas = df[colunas_utilizadas]

# No pandas a gente pode selecionar uma coluna, aplicar uma função e salvar o resultado em uma nuva coluna;
#   para aplicar a função usamos a função apply()

def string_para_minusculo(valor):
    return str(valor).lower()

@np.vectorize
def vec_string_para_minusculo(valor):
    return str(valor).lower()

df["season_minusculo"] = df["Season"].apply(string_para_minusculo)

# Também podemos substituir determinados valores usando a função replace()

df["season_minusculo"] = df["season_minusculo"].replace({"autumn":"automn"})

rotulos_string = df["season_minusculo"]

# Nossa coluna de rótulos (labels em Inglês) tem Strings, mas precisamos que sejam números para usar os algoritmos
#   do scikit-learn
# Vamos converter as strings para números usando a classe LabelEncoder() do scikit-learn
#   Essa função transformar cada string em uma classe de 0 n_classes-1
#   Exemplo: ["vermelho","azul","azul"] se torna [0,1,1]

rotulos_encoder = preprocessing.LabelEncoder()

# Instanciamos a classe LabelEncoder() acima, mas para transformar os dados precisamos que a classe:
#   1. aprenda qual vai ser o número de cada string: método fit()
#   2. transforme os dados em números: método transform
rotulos_encoder.fit(rotulos_string)

rotulos = rotulos_encoder.transform(rotulos_string)

# O scikit-learn possui uma função chamada train_test_split() que divide um conjunto de dados em conjunto de treinamento
#   e conjunto de testes. Isso é importante porque sem o conjunto de testes a gente não vão poder avaliar se um modelo
#   apresenta bons resultados ou não
# Vamos usar o paramêtro stratify para que os dois conjuntos tenham a mesma proporção de classes

dados_treinamento, dados_teste, rotulos_treinamento, rotulos_teste = train_test_split(df_colunas_utilizadas,
                                                                                      rotulos,
                                                                                      stratify=rotulos,
                                                                                      test_size=0.33,
                                                                                      random_state=42)

dados_treinamento_sem_valores_faltantes = dados_treinamento.fillna("-1")

# Strings para números com o OneHotEncoding
dados_treinamento_minusculo = dados_treinamento_sem_valores_faltantes.apply(vec_string_para_minusculo,axis="index")

one_hot_encoder = OneHotEncoder(handle_unknown='ignore')

one_hot_encoder.fit(dados_treinamento_minusculo)

dados_treinamento_transformados_one_hot = one_hot_encoder.transform(dados_treinamento_minusculo)

FEATURE_NAMES = one_hot_encoder.get_feature_names()

# A gente pode escolher as features com mais poder preditivo
#   Vamos usar a classe SelectKBest do scikit-learn para fazer isso e a função chi2 para calcular os scores

seletor_de_features = SelectKBest(chi2, k=20)
dados_treinamento_features_selecionadas = seletor_de_features.fit_transform(dados_treinamento_transformados_one_hot, rotulos_treinamento)
features_selecionadas = FEATURE_NAMES[seletor_de_features.get_support()]

# Treinando o modelo
clf = DecisionTreeClassifier(max_depth=7, random_state=50)
clf = clf.fit(dados_treinamento_features_selecionadas, rotulos_treinamento)

## 1. Avaliando o modelo

In [0]:
# As mesmas etapas que fizemos nas features de treinamento
dados_teste_sem_valores_faltantes = dados_teste.fillna("-1")

dados_teste_transformados = one_hot_encoder.transform(dados_teste_sem_valores_faltantes)

dados_teste_features_selecionadas = seletor_de_features.transform(dados_teste_transformados)

# acurácia média
#   https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html

clf.score(dados_teste_features_selecionadas,rotulos_teste)

0.40606060606060607

In [0]:
# Salvando as predições em uma variável
predicoes = clf.predict(dados_teste_features_selecionadas)

In [0]:
predicoes

array([2, 3, 3, 1, 3, 2, 2, 2, 2, 1, 2, 3, 1, 0, 2, 3, 1, 1, 2, 2, 3, 2,
       3, 1, 1, 1, 3, 2, 1, 3, 2, 3, 1, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1,
       1, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 1, 3, 3, 1, 3, 2, 1, 2, 1, 1,
       2, 2, 1, 1, 2, 2, 3, 2, 2, 2, 3, 1, 3, 2, 2, 3, 1, 3, 3, 1, 1, 1,
       1, 3, 3, 2, 2, 1, 2, 2, 2, 1, 3, 1, 1, 3, 2, 2, 3, 2, 2, 2, 1, 2,
       1, 2, 1, 2, 1, 2, 3, 0, 2, 2, 2, 2, 3, 1, 2, 1, 1, 2, 2, 2, 2, 2,
       1, 2, 1, 2, 1, 2, 3, 3, 3, 3, 1, 1, 0, 2, 2, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 1, 3, 1, 2, 3, 1, 2, 2])

In [0]:
# Qual a distribuição das classes no conjunto de testes?

rotulos_teste_string = rotulos_encoder.inverse_transform(rotulos_teste)
df_rotulos_teste_string = pd.DataFrame(rotulos_teste_string,columns=["classe"])
display(df_rotulos_teste_string["classe"].value_counts(dropna=False))

summer    53
winter    48
spring    41
automn    23
Name: classe, dtype: int64

In [0]:
# Quanto em %?
#   Isso é o class imbalance (classes com muito mais instâncias que as outras)
display(df_rotulos_teste_string["classe"].value_counts(normalize=True)*100)

summer    32.121212
winter    29.090909
spring    24.848485
automn    13.939394
Name: classe, dtype: float64

In [0]:
# Se a gente disser que todas as predições são summer, qual o score?
from sklearn.metrics import accuracy_score

accuracy_score(rotulos_encoder.transform(np.array(["summer"]*165)), rotulos_teste)

0.3212121212121212

(ir para o slide falar sobre as métricas)

In [0]:
rotulos_encoder.classes_

array(['automn', 'nan', 'spring', 'summer', 'winter'], dtype=object)

In [0]:
from sklearn.metrics import classification_report

print(classification_report(predicoes, rotulos_teste))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.29      0.26      0.27        47
           2       0.68      0.44      0.54        81
           3       0.40      0.56      0.46        34

    accuracy                           0.41       165
   macro avg       0.34      0.31      0.32       165
weighted avg       0.50      0.41      0.44       165



### 1.1 Vendo a probabilidade de cada classe para uma linha dos dados

In [0]:
# selecionamos a linha zero assim: [0,:]
pd.DataFrame(dados_teste_features_selecionadas[0,:].toarray(),columns=features_selecionadas)

,x0_brief,x0_cute,x0_flare,x0_party,x0_sexy,x1_scoop,x2_halfsleeve,x2_threequarter,x3_cotton,x3_lace,x3_linen,x3_other,x3_polyster,x3_rayon,x3_shiffon,x3_silk,x4_chiffon,x4_jersey,x5_-1,x5_feathers
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
# utilizamos o método predict_proba() para ver o score para cada uma das classes
pd.DataFrame(clf.predict_proba(dados_teste_features_selecionadas[0,:]), columns=rotulos_encoder.classes_)

,automn,spring,summer,winter
0,0.103896,0.194805,0.532468,0.168831


In [0]:
# a classe é atribuída ao maior score (que no caso acima é 'summer')
rotulos_encoder.inverse_transform([rotulos_teste[0]])

array(['summer'], dtype=object)

### 1.2 Consertando o 'nan' nas classes
Lá em cima, após carregar os dados do CSV, vamos remover as linhas que possuem valor 'nan' na coluna 'Season'

`df = df.dropna(subset=["Season"])`

In [0]:
df["season_minusculo"].value_counts(dropna=False)

summer    160
winter    145
spring    124
automn     69
nan         2
Name: season_minusculo, dtype: int64

Agora temos que rodar novamente os códigos que usamos para testar o modelo

### 1.3 Onde o modelo tá errando?
Vamos ver com a **matriz de confusão** (confusion matrix).
Cada linha representa a classe correta. O número representa a quantidade de instâncias da classe do índice que foram classificadas na classe da coluna.

In [0]:
from sklearn.metrics import confusion_matrix

pd.DataFrame(confusion_matrix(predicoes, rotulos_teste),columns=rotulos_encoder.classes_,index=rotulos_encoder.classes_)

,automn,spring,summer,winter
automn,0,1,1,1
spring,12,12,11,12
summer,8,21,36,16
winter,3,7,5,19


Ao observar a matriz de confusão acima vemos que nenhuma instância foi corretamente classificada como 'automn', e o maior número de erros tá na classe 'summer' que foi classificada como 'spring' 21 vezes.

Vamos olhar os casos onde o modelo está errando:

In [0]:
df_corretas_predicoes = pd.DataFrame(rotulos_teste,columns=["correta"]).join(pd.DataFrame(predicoes,columns=["predicao"]))
df_corretas_predicoes.query('correta != predicao').index

Int64Index([  1,   3,   5,   7,   8,  10,  11,  12,  13,  16,  18,  20,  21,
             23,  24,  26,  28,  29,  30,  32,  34,  35,  38,  42,  43,  44,
             46,  47,  48,  49,  52,  53,  55,  59,  61,  62,  63,  64,  65,
             67,  68,  71,  74,  76,  77,  78,  82,  86,  87,  89,  90,  91,
             93,  94,  97,  98, 101, 105, 107, 108, 109, 110, 113, 114, 117,
            118, 119, 120, 121, 123, 124, 125, 126, 130, 131, 133, 134, 136,
            137, 140, 143, 144, 146, 147, 148, 149, 150, 152, 154, 155, 156,
            157, 158, 159, 161, 162, 163, 164],
           dtype='int64')

In [0]:
index = 1

display(pd.DataFrame(dados_teste_features_selecionadas[index,:].toarray(),columns=features_selecionadas))
display(pd.DataFrame(clf.predict_proba(dados_teste_features_selecionadas[index,:]), columns=rotulos_encoder.classes_))
print(df_corretas_predicoes.iloc[index])

,x0_brief,x0_cute,x0_flare,x0_party,x0_sexy,x1_scoop,x2_halfsleeve,x2_threequarter,x3_cotton,x3_lace,x3_linen,x3_other,x3_polyster,x3_rayon,x3_shiffon,x3_silk,x4_chiffon,x4_jersey,x5_-1,x5_feathers
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


,automn,spring,summer,winter
0,0.0,0.0,0.0,1.0


correta     1
predicao    3
Name: 1, dtype: int64


In [0]:
rotulos_encoder.classes_

array(['automn', 'spring', 'summer', 'winter'], dtype=object)

In [0]:
index = 2

display(pd.DataFrame(dados_teste_features_selecionadas[index,:].toarray(),columns=features_selecionadas))
display(pd.DataFrame(clf.predict_proba(dados_teste_features_selecionadas[index,:]), columns=rotulos_encoder.classes_))
print(df_corretas_predicoes.iloc[index])

,x0_brief,x0_cute,x0_flare,x0_party,x0_sexy,x1_scoop,x2_halfsleeve,x2_threequarter,x3_cotton,x3_lace,x3_linen,x3_other,x3_polyster,x3_rayon,x3_shiffon,x3_silk,x4_chiffon,x4_jersey,x5_-1,x5_feathers
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


,automn,spring,summer,winter
0,0.0,0.333333,0.0,0.666667


correta     3
predicao    3
Name: 2, dtype: int64


### 1.4 Quais os melhores parâmetros para a Árvore de Decisão?

In [0]:
from sklearn.model_selection import GridSearchCV

parameters = {'criterion':('gini', 'entropy'), 'splitter':["best", "random"], "max_depth":[3,5,7,10]}

clf_grid = GridSearchCV(clf, parameters)
clf_grid.fit(dados_treinamento_features_selecionadas, rotulos_treinamento)


clf_grid.best_params_

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'criterion': 'entropy', 'max_depth': 3, 'splitter': 'best'}

Os objetos das classes GridSearchCV e RandomSearchCV também se comportam como Estimators (ver slides da Sessão 01 sobre a API do Scikit-learn). Isso quer dizer que podemos chamar o método .fit() e ele utiliza o modelo com os melhores parâmetros para fazer as predições.

Vamos comparar o desempenho com o desempenho do modelo que treinamos antes:

In [0]:
# Salvando as predições em uma variável
predicoes_grid = clf_grid.predict(dados_teste_features_selecionadas)
print(classification_report(predicoes_grid, rotulos_teste))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.34      0.31      0.33        45
           2       0.74      0.43      0.54        91
           3       0.38      0.62      0.47        29

    accuracy                           0.43       165
   macro avg       0.36      0.34      0.33       165
weighted avg       0.56      0.43      0.47       165



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [0]:
df_corretas_predicoes = pd.DataFrame(rotulos_teste,columns=["correta"]).join(pd.DataFrame(predicoes_grid,columns=["predicao"]))
df_corretas_predicoes.query('correta != predicao').index

Int64Index([  2,   3,   5,   7,   8,  10,  11,  12,  13,  16,  17,  18,  21,
             23,  25,  28,  29,  30,  32,  34,  35,  38,  42,  43,  46,  47,
             48,  49,  53,  55,  59,  61,  62,  63,  64,  65,  67,  68,  69,
             71,  72,  76,  77,  78,  82,  86,  87,  89,  90,  91,  93,  94,
             97,  98,  99, 101, 105, 107, 108, 109, 110, 113, 114, 117, 118,
            119, 120, 121, 123, 124, 125, 126, 130, 131, 133, 134, 136, 137,
            140, 143, 146, 147, 148, 149, 150, 152, 154, 155, 156, 157, 161,
            162, 163, 164],
           dtype='int64')